# Reverse-geocode lat-long to census geography

Using https://geocoding.geo.census.gov/

In [1]:
import time, pandas as pd, requests

### Define helper functions

In [2]:
def census_geocode(lon, lat, benchmark='Public_AR_Current', vintage='Current_Current',
                   output_format='json', layers='Census Tracts'):
    
    url = ('https://geocoding.geo.census.gov/geocoder/geographies/coordinates?'
           'benchmark={benchmark}&vintage={vintage}&x={lon}&y={lat}&format={output_format}&layers={layers}')
    
    prepared_url = url.format(benchmark=benchmark, vintage=vintage, lon=lon, lat=lat, 
                              output_format=output_format, layers=layers)

    response = requests.get(prepared_url)
    return response

In [3]:
def geocode_row(row):

    response = census_geocode(lon=row['longitude'], lat=row['latitude'])
    result = response.json()['result']
    tract = result['geographies']['Census Tracts'][0]
    
    return pd.Series(tract)

### Load the data and reverse geocode it

In [4]:
# load the dataset of rental listings
listings = pd.read_csv('data/listings.csv')

In [5]:
# reverse geocode each listing's lat-long to tract
tracts = listings.apply(lambda row: geocode_row(row), axis=1)

In [6]:
# look at the tracts we got back
tracts[['GEOID', 'AREALAND', 'AREAWATER']].head()

,GEOID,AREALAND,AREAWATER
0,06083001304,5090828,144119
1,51153900701,2702493,43468
2,24031705000,4552733,14831
3,06073016301,1068848,0
4,22063040806,69992596,1310597


In [7]:
# merge the listings with the tracts' geoids
geoids = pd.DataFrame(tracts['GEOID'])
listings_geoids = pd.concat([listings, geoids], axis=1)
listings_geoids.head()

,pid,date,region,neighborhood,rent,bedrooms,sqft,rent_sqft,rent_sqft_cat,longitude,latitude,GEOID
0,4454264047,2014-05-11,santabarbara,NaN,3500.0,3.0,1200.0,2.916667,5,-119.726987,34.399757,06083001304
1,4468128892,2014-05-13,washingtondc,14300 Jeffries Rd Ste 1207,1099.0,2.0,775.0,1.418065,4,-77.268300,38.635600,51153900701
2,4474037012,2014-05-16,washingtondc,Bethesda,2743.0,1.0,714.0,3.841737,5,-77.102200,39.000300,24031705000
3,4482003715,2014-05-21,sandiego,escondido,995.0,1.0,614.0,1.620521,4,-116.950989,32.807693,06073016301
4,4488888498,2014-05-26,batonrouge,Denham Springs,1000.0,3.0,1100.0,0.909091,2,-90.942121,30.438018,22063040806
